In [51]:
from neo4jrestclient.client import GraphDatabase
from neo4jrestclient import client
from bs4 import BeautifulSoup
from collections import defaultdict

In [2]:
db = GraphDatabase("http://localhost:7474", username="neo4j", password="sage")


In [37]:
def get_person_node(name):
    try:
        q = """Match (n:Person) where n.name="{}" RETURN n""".format(name)
        results = db.query(q, returns=(client.Node, client.Relationship))
        return results[0]
    except:
        return None

In [46]:
def get_keyword_node(name):
    try:
        q = """Match (n:Keyword) where n.name="{}" RETURN n""".format(name)
        results = db.query(q, returns=(client.Node, client.Relationship))
        return results[0]
    except:
        return None

In [52]:
# check we can access the file and process it with Beautiful soup 
filename = "accepted_talks.html"
html = open(filename,"r").read()
soup = BeautifulSoup(html)

/Users/ianm/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/ianm/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [53]:
talks = []
section = soup.find('section', {'class': 'section', 'id': 'colgne'})
div = section.find("div")
btags = div.findAll("b")
for btag in btags:
    title = btag.contents
    authors = btag.next_sibling.next_sibling
    author_list = authors.replace(" and ", ",").split(",") 
    talks.append((title, author_list))

In [54]:
# now we have extracted the titles and authors from the page, I noticed that the talks are listed twice on the page, 
# lets get a unique set of talks
unique_talks = []
for talk in talks:
    if talk not in unique_talks:
        unique_talks.append(talk)

In [56]:
# get a dumb set of stopwords
stopwords = open("SmartStoplist.txt").readlines()
stopwords_stripped = list(map(lambda x: x.strip(), stopwords))
strip_characters = [":", "?", "-", "'", "`", ";", ",", ".", "\"" ]

In [57]:
def clean_word(word):
    """
    Ok, we are going to do a bit of claning up here, just a quick job, namely
    normalise words to lowercase 
    get rid of some punctuation 
    if the word is not on our list, strip s from the end to get rid of plural variants (so dumb, but works right now)
    """
    candidate_word = word.lower().strip()
    current_word = candidate_word
    for char in strip_characters:
        current_word = current_word.replace(char, '')
    if current_word not in ["bias", "embeddedne", "ubiquitous", "analysis", "process", "strategie", "properties"]:
        cleaned_word = current_word.rstrip("s") # dumb way of getting rid of plurals, OK for a short analysis 
        return cleaned_word
    else:
        return current_word

In [58]:
def get_keywords(title):
    """
    go through all of the words in a titls, and drop words in our small stop word list
    that way we can try to generate words that might look at bit like keywords for the talks. 
    """
    keywords = []
    for word in title.split():
        cleaned_word = clean_word(word)
        if cleaned_word not in stopwords_stripped:
            keywords.append(cleaned_word)
    return keywords

In [66]:
# now check what that looks like against our titles 
author_keywords = []
for talk in unique_talks:
    title = talk[0][0]
    authors = talk[1]
    keywords = get_keywords(title)
    print(title, keywords) # looks pretty good
    print(authors)
    for author in authors:
        author_node = get_person_node(author.strip())
        print(node)   
        for keyword in keywords:
            keyword_node = get_keyword_node(keyword)
            print(node)
            author_keyword = (author_node, keyword_node)
            if author_keyword not in author_keywords:
                author_keywords.append(author_keyword)

School segregation in the digital space ['school', 'segregation', 'digital', 'space']
['Ivan Smirnov']
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
Bounded confidence in extreme opinion evolution : an online experiment ['bounded', 'confidence', 'extreme', 'opinion', 'evolution', '', 'online', 'experiment']
['Samuel Martin', ' Sylvie Huet', ' Pascal Gend', ' Mohammed Amine Ait Oumajoud', ' Guillaume Deffuant', ' Armelle Nugier', 'Serge Guimond']
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/no

[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j No

[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
Sampling from Social Networks with Attributes ['sampling', 'social', 'network', 'attribute']
['Claudia Wagner', ' Philipp Singer', ' Fariba Karimi', ' Jürgen Pfeffer', 'Markus Strohmaier']
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/da

[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j No

[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j No

...

[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
Opportunities and Challenges in Computational Social Science: A Publisher’s Perspective ['opportunitie', 'challenge', 'computational', 'social', 'science', 'publisher’', 'perspective']
['Katie Metzler', 'Ian Mulvany']
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node

[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
Evolution of Moral Politics: Analysis on the Moral Foundations of Korean Political Parties ['evolution', 'moral', 'politic', 'analysis', 'moral', 'foundation', 'korean', 'political', 'partie']
['Changdong Oh']
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
Small cities face greater impact from automation ['small', 'citie', 'face',

[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j No

[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
Women and Creativity: Gender Differences in the Production of Popular Music ['women', 'creativity', 'gender', 'difference', 'production', 'popular', 'music']
['Michael Mauskap

[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
[<Neo4j Node: http://localhost:7474/db/data/node/415>]
What comes first? Social strength or common friends? ['social', '

In [73]:
# create the link between people and topics of interest 
for ak in author_keywords:
    ak[0][0].Studies(ak[1][0])
    print(ak[0][0], ak[1][0])

<Neo4j Node: http://localhost:7474/db/data/node/618> <Neo4j Node: http://localhost:7474/db/data/node/1>
<Neo4j Node: http://localhost:7474/db/data/node/618> <Neo4j Node: http://localhost:7474/db/data/node/2>
<Neo4j Node: http://localhost:7474/db/data/node/618> <Neo4j Node: http://localhost:7474/db/data/node/3>
<Neo4j Node: http://localhost:7474/db/data/node/618> <Neo4j Node: http://localhost:7474/db/data/node/4>
<Neo4j Node: http://localhost:7474/db/data/node/619> <Neo4j Node: http://localhost:7474/db/data/node/5>
<Neo4j Node: http://localhost:7474/db/data/node/619> <Neo4j Node: http://localhost:7474/db/data/node/6>
<Neo4j Node: http://localhost:7474/db/data/node/619> <Neo4j Node: http://localhost:7474/db/data/node/7>
<Neo4j Node: http://localhost:7474/db/data/node/619> <Neo4j Node: http://localhost:7474/db/data/node/8>
<Neo4j Node: http://localhost:7474/db/data/node/619> <Neo4j Node: http://localhost:7474/db/data/node/9>
<Neo4j Node: http://localhost:7474/db/data/node/619> <Neo4j Node

<Neo4j Node: http://localhost:7474/db/data/node/632> <Neo4j Node: http://localhost:7474/db/data/node/14>
<Neo4j Node: http://localhost:7474/db/data/node/632> <Neo4j Node: http://localhost:7474/db/data/node/31>
<Neo4j Node: http://localhost:7474/db/data/node/633> <Neo4j Node: http://localhost:7474/db/data/node/25>
<Neo4j Node: http://localhost:7474/db/data/node/633> <Neo4j Node: http://localhost:7474/db/data/node/26>
<Neo4j Node: http://localhost:7474/db/data/node/633> <Neo4j Node: http://localhost:7474/db/data/node/27>
<Neo4j Node: http://localhost:7474/db/data/node/633> <Neo4j Node: http://localhost:7474/db/data/node/28>
<Neo4j Node: http://localhost:7474/db/data/node/633> <Neo4j Node: http://localhost:7474/db/data/node/29>
<Neo4j Node: http://localhost:7474/db/data/node/633> <Neo4j Node: http://localhost:7474/db/data/node/23>
<Neo4j Node: http://localhost:7474/db/data/node/633> <Neo4j Node: http://localhost:7474/db/data/node/30>
<Neo4j Node: http://localhost:7474/db/data/node/633> <N

<Neo4j Node: http://localhost:7474/db/data/node/651> <Neo4j Node: http://localhost:7474/db/data/node/61>
<Neo4j Node: http://localhost:7474/db/data/node/651> <Neo4j Node: http://localhost:7474/db/data/node/62>
<Neo4j Node: http://localhost:7474/db/data/node/651> <Neo4j Node: http://localhost:7474/db/data/node/63>
<Neo4j Node: http://localhost:7474/db/data/node/652> <Neo4j Node: http://localhost:7474/db/data/node/64>
<Neo4j Node: http://localhost:7474/db/data/node/652> <Neo4j Node: http://localhost:7474/db/data/node/65>
<Neo4j Node: http://localhost:7474/db/data/node/652> <Neo4j Node: http://localhost:7474/db/data/node/66>
<Neo4j Node: http://localhost:7474/db/data/node/652> <Neo4j Node: http://localhost:7474/db/data/node/67>
<Neo4j Node: http://localhost:7474/db/data/node/652> <Neo4j Node: http://localhost:7474/db/data/node/68>
<Neo4j Node: http://localhost:7474/db/data/node/652> <Neo4j Node: http://localhost:7474/db/data/node/69>
<Neo4j Node: http://localhost:7474/db/data/node/652> <N

<Neo4j Node: http://localhost:7474/db/data/node/672> <Neo4j Node: http://localhost:7474/db/data/node/87>
<Neo4j Node: http://localhost:7474/db/data/node/673> <Neo4j Node: http://localhost:7474/db/data/node/82>
<Neo4j Node: http://localhost:7474/db/data/node/673> <Neo4j Node: http://localhost:7474/db/data/node/83>
<Neo4j Node: http://localhost:7474/db/data/node/673> <Neo4j Node: http://localhost:7474/db/data/node/84>
<Neo4j Node: http://localhost:7474/db/data/node/673> <Neo4j Node: http://localhost:7474/db/data/node/85>
<Neo4j Node: http://localhost:7474/db/data/node/673> <Neo4j Node: http://localhost:7474/db/data/node/86>
<Neo4j Node: http://localhost:7474/db/data/node/673> <Neo4j Node: http://localhost:7474/db/data/node/87>
<Neo4j Node: http://localhost:7474/db/data/node/674> <Neo4j Node: http://localhost:7474/db/data/node/88>
<Neo4j Node: http://localhost:7474/db/data/node/674> <Neo4j Node: http://localhost:7474/db/data/node/89>
<Neo4j Node: http://localhost:7474/db/data/node/674> <N

TypeError: 'NoneType' object is not subscriptable